# Recommender Systems Tutorial
In this tutorial, we'll provide a simple walkthrough of how to use Snorkel to improve recommendations. We consider a setting similar to the [Netflix challenge](https://www.kaggle.com/netflix-inc/netflix-prize-data), but with books instead of movies. We have a set of users and books, and for each user we know the set of books they have interacted with (read or marked as to-read). We don't have the user's ratings for the read books, except in a small number of cases. We also have some text reviews written by users. Our goal is to predict whether a user will read and like any given book. We represent users using the set of books they have interacted with, and train a model to predict a `rating` given the set of books the user interacted with and a new book to be rated (a `rating` of 1 means the user will read and like the book). Note that [Recommender systems](https://en.wikipedia.org/wiki/Recommender_system) is a very well studied area with a wide variety of settings and approaches, and we just focus on one of them.

We will use the [Goodreads](https://sites.google.com/eng.ucsd.edu/ucsdbookgraph/home) dataset, from
"Item Recommendation on Monotonic Behavior Chains", RecSys'18 (Mengting Wan, Julian McAuley), and "Fine-Grained Spoiler Detection from Large-Scale Review Corpora", ACL'19 (Mengting Wan, Rishabh Misra, Ndapa Nakashole, Julian McAuley).
In this dataset, we have user interactions and reviews for Young Adult novels from the Goodreads website, along with metadata (like `title` and `authors`) for the novels.

In [1]:
import os

if os.path.basename(os.getcwd()) == "snorkel-tutorials":
    os.chdir("recsys")

## Loading Data

We start by running the `download_and_process_data` function. The function returns the `df_train`, `df_test`, `df_dev`, `df_val` dataframes, which correspond to our training, test, development, and validation sets. Each of those dataframes has the following fields:
* `user_idx`: A unique identifier for a user.
* `book_idx`: A unique identifier for a book that is being rated by the user.
* `book_idxs`: The set of books that the user has interacted with (read or planned to read).
* `review_text`: Optional text review written by the user for the book.
* `rating`: Either `0` (which means the user did not read or did not like the book) or `1` (which means the user read and liked the book). The `rating` field is missing for `df_train`.
Our objective is to predict whether a given user (represented by the set of book_idxs the user has interacted with) will read and like any given book. That is, we want to train a model that takes a set of `book_idxs` (the user) and a single `book_idx` (the book to rate) and predicts the `rating`.

In addition, `download_and_process_data` also returns the `df_books` dataframe, which contains one row per book, along with metadata for that book (such as `title` and `first_author`).

In [2]:
from utils import download_and_process_data

(df_train, df_test, df_dev, df_val), df_books = download_and_process_data()

df_books.head()

/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/snorkel-t

/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

/home/ubuntu/snorkel-tutorials/recsys/utils.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_interactions_nz["rating_4_5"] = df_interactions_nz.rating.map(ratings_map)


,authors,average_rating,book_id,country_code,description,is_ebook,language_code,ratings_count,similar_books,text_reviews_count,title,first_author,book_idx
3,[293603],4.35,10099492,US,It all comes down to this.\nVlad's running out...,True,eng,152,"[25861113, 7430195, 18765937, 6120544, 3247550...",9,Twelfth Grade Kills (The Chronicles of Vladimi...,293603,0
4,[4018722],3.71,22642971,US,The future world is at peace.\nElla Shepherd h...,True,eng,1525,"[20499652, 17934493, 13518102, 16210411, 17149...",428,The Body Electric,4018722,1
5,[6537142],3.89,31556136,US,A gorgeously written and deeply felt literary ...,True,,109,[],45,Like Water,6537142,2
12,"[6455200, 5227552]",3.90,18522274,US,Zoe Vanderveen is on the run with her captor t...,True,en-US,191,"[25063023, 18553080, 17567752, 18126509, 17997...",6,"Volition (The Perception Trilogy, #2)",6455200,3
13,[187837],3.19,17262776,US,"The war is over, but for thirteen-year-old Rac...",True,eng,248,"[16153997, 10836616, 17262238, 16074827, 13628...",68,Little Red Lies,187837,4


We look at a sample of the labeled development set. As an example, we want our final recommendations model to be able to predict that a user who has interacted with `book_idxs` (25743, 22318, 7662, 6857, 83, 14495, 30664, ...) would either not read or not like the book with `book_idx` 22764 (first row), while a user who has interacted with `book_idxs` (3880, 18078, 9092, 29933, 1511, 8560, ...) would read and like the book with `book_idx` 3181 (second row).

In [3]:
df_dev.sample(frac=1, random_state=12).head()

,user_idx,book_idxs,book_idx,rating,review_text
80480,3115,"(30671, 21132, 20057, 16271, 8224, 10340, 9046...",4165,1,NaN
80483,3115,"(30671, 21132, 20057, 16271, 8224, 10340, 9046...",27119,1,NaN
782352,30443,"(1696, 10119, 2486, 14371, 17962, 2081, 1742, ...",20471,0,NaN
538899,20809,"(3854, 13950, 17535, 12642, 15803, 8199, 28827...",27766,1,NaN
768968,29931,"(1325, 14324, 19639, 1511, 19219, 16980, 4363,...",17807,0,NaN


## Writing Labeling Functions

If a user has interacted with several books written by an author, there is a good chance that the user will read and like other books by the same author. We express this as a labeling function, using the `first_author` field in the `df_books` dataframe.

In [4]:
POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1

In [5]:
from snorkel.labeling.lf import labeling_function

book_to_first_author = dict(zip(df_books.book_idx, df_books.first_author))
first_author_to_books_df = df_books.groupby("first_author")[["book_idx"]].agg(set)
first_author_to_books = dict(
    zip(first_author_to_books_df.index, first_author_to_books_df.book_idx)
)


@labeling_function()
def shared_first_author(x):
    author = book_to_first_author[x.book_idx]
    same_author_books = first_author_to_books[author]
    num_read = len(set(x.book_idxs).intersection(same_author_books))
    return POSITIVE if num_read > 15 else ABSTAIN

We can also leverage the long text reviews written by users to guess whether they liked or disliked a book. For example, the third df_dev entry above has a review with the text '4.5 STARS', which indicates that the user liked the book. We write a simple LF that looks for similar phrases to guess the user's rating of a book. We interpret >= 4 stars to indicate a positive rating, while < 4 stars is negative.

In [6]:
low_rating_strs = [
    "one star",
    "1 star",
    "two star",
    "2 star",
    "3 star",
    "three star",
    "3.5 star",
    "2.5 star",
    "1 out of 5",
    "2 out of 5",
    "3 out of 5",
]
high_rating_strs = ["5 stars", "five stars", "four stars", "4 stars", "4.5 stars"]


@labeling_function()
def stars_in_review(x):
    if not isinstance(x.review_text, str):
        return ABSTAIN
    for low_rating_str in low_rating_strs:
        if low_rating_str in x.review_text.lower():
            return NEGATIVE
    for high_rating_str in high_rating_strs:
        if high_rating_str in x.review_text.lower():
            return POSITIVE
    return ABSTAIN

We can also run [TextBlob](https://textblob.readthedocs.io/en/dev/index.html), a tool that provides a pretrained sentiment analyzer, on the reviews, and use its polarity and subjectivity scores to estimate the user's rating for the book.

In [7]:
from snorkel.preprocess import preprocessor
from textblob import TextBlob


@preprocessor(memoize=True)
def textblob_polarity(x):
    if x.review_text:
        x.blob = TextBlob(str(x.review_text))
    else:
        x.blob = None
    return x


# Label high polarity reviews as positive.
@labeling_function(pre=[textblob_polarity])
def polarity_positive(x):
    if x.blob:
        if x.blob.polarity > 0.3:
            return POSITIVE
    return ABSTAIN


# Label high subjectivity reviews as positive.
@labeling_function(pre=[textblob_polarity])
def subjectivity_positive(x):
    if x.blob:
        if x.blob.subjectivity > 0.75:
            return POSITIVE
    return ABSTAIN


# Label low polarity reviews as negative.
@labeling_function(pre=[textblob_polarity])
def polarity_negative(x):
    if x.blob:
        if x.blob.polarity < 0.0:
            return NEGATIVE
    return ABSTAIN

In [8]:
from snorkel.labeling import PandasLFApplier, LFAnalysis

lfs = [
    stars_in_review,
    shared_first_author,
    polarity_positive,
    subjectivity_positive,
    polarity_negative,
]

applier = PandasLFApplier(lfs)
L_dev = applier.apply(df_dev)
LFAnalysis(L_dev, lfs).lf_summary(df_dev.rating)

  0%|          | 0/84237 [00:00<?, ?it/s]

  0%|          | 40/84237 [00:00<03:32, 396.79it/s]

  0%|          | 110/84237 [00:00<03:04, 455.25it/s]

  0%|          | 186/84237 [00:00<02:42, 517.34it/s]

  0%|          | 264/84237 [00:00<02:26, 574.05it/s]

...

100%|█████████▉| 84063/84237 [02:14<00:00, 663.61it/s]

100%|█████████▉| 84141/84237 [02:14<00:00, 694.46it/s]

100%|█████████▉| 84213/84237 [02:15<00:00, 659.50it/s]

100%|██████████| 84237/84237 [02:15<00:00, 623.23it/s]

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
stars_in_review,0,"[0, 1]",0.018614,0.004547,0.001793,1275,293,0.813138
shared_first_author,1,[1],0.043591,0.000807,0.000249,2793,879,0.760621
polarity_positive,2,[1],0.045835,0.013035,0.000962,3123,738,0.808858
subjectivity_positive,3,[1],0.017249,0.013082,0.002766,1093,360,0.752237
polarity_negative,4,[0],0.015836,0.003953,0.003110,727,607,0.544978


### Applying labeling functions to the training set

We apply the labeling functions to the training set, and then filter out examples unlabeled by any LF, and combine the rest with the dev set to form our final training set.

In [9]:
from snorkel.labeling.model.label_model import LabelModel

# Train LabelModel.
L_train = applier.apply(df_train)
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=5000, seed=123, log_freq=20, lr=0.01)
Y_train_preds = label_model.predict(L_train)

  0%|          | 0/600658 [00:00<?, ?it/s]

  0%|          | 3/600658 [00:00<5:34:57, 29.89it/s]

  0%|          | 75/600658 [00:00<3:58:37, 41.95it/s]

  0%|          | 145/600658 [00:00<2:51:20, 58.41it/s]

  0%|          | 211/600658 [00:00<2:04:29, 80.38it/s]

...

100%|█████████▉| 600434/600658 [15:55<00:00, 472.84it/s]

100%|█████████▉| 600506/600658 [15:55<00:00, 526.63it/s]

100%|█████████▉| 600572/600658 [15:55<00:00, 551.33it/s]

100%|█████████▉| 600646/600658 [15:55<00:00, 595.69it/s]

100%|██████████| 600658/600658 [15:55<00:00, 628.48it/s]

In [10]:
import pandas as pd
from snorkel.labeling import filter_unlabeled_dataframe

df_train_filtered, Y_train_preds_filtered = filter_unlabeled_dataframe(
    df_train, Y_train_preds, L_train
)
df_train_filtered["rating"] = Y_train_preds_filtered
combined_df_train = pd.concat([df_train_filtered, df_dev], axis=0)

/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


### Rating Prediction Model
We write a Keras model for predicting ratings given a user's book list and a book (which is being rated). The model represents the list of books the user interacted with, `books_idxs`, by learning an embedding for each idx, and averaging the embeddings in `book_idxs`. It learns another embedding for the `book_idx`, the book to be rated. Then it concatenates the two embeddings and uses an [MLP](https://en.wikipedia.org/wiki/Multilayer_perceptron) to compute the probability of the `rating` being 1.

In [11]:
import numpy as np
import tensorflow as tf
from utils import precision, recall, f1

n_books = len(df_books)


# Keras model to predict rating given book_idxs and book_idx.
def get_model(embed_dim=64, hidden_layer_sizes=[32]):
    # Compute embedding for book_idxs.
    len_book_idxs = tf.keras.layers.Input([])
    book_idxs = tf.keras.layers.Input([None])
    book_idxs_emb = tf.keras.layers.Embedding(n_books, embed_dim)(book_idxs)
    book_idxs_emb = tf.math.divide(
        tf.keras.backend.sum(book_idxs_emb, axis=1), tf.expand_dims(len_book_idxs, 1)
    )
    # Compute embedding for book_idx.
    book_idx = tf.keras.layers.Input([])
    book_idx_emb = tf.keras.layers.Embedding(n_books, embed_dim)(book_idx)
    input_layer = tf.keras.layers.concatenate([book_idxs_emb, book_idx_emb], 1)
    # Build Multi Layer Perceptron on input layer.
    cur_layer = input_layer
    for size in hidden_layer_sizes:
        tf.keras.layers.Dense(size, activation=tf.nn.relu)(cur_layer)
    output_layer = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(cur_layer)
    # Create and compile keras model.
    model = tf.keras.Model(
        inputs=[len_book_idxs, book_idxs, book_idx], outputs=[output_layer]
    )
    model.compile(
        "Adagrad", "binary_crossentropy", metrics=["accuracy", f1, precision, recall]
    )
    return model

We use triples of (`book_idxs`, `book_idx`, `rating`) from our dataframes as training examples. In addition, we want to train the model to recognize when a user will not read a book. To create examples for that, we randomly sample a `book_id` not in `book_idxs` and use that with a `rating` of 0 as a _random negative_ example. We create one such _random negative_ example for every positive (`rating` 1) example in our dataframe so that positive and negative examples are roughly balanced.

In [12]:
# Generator to turn dataframe into examples.
def get_examples_generator(df):
    def generator():
        for book_idxs, book_idx, rating in zip(df.book_idxs, df.book_idx, df.rating):
            # Remove book_idx from book_idxs so the model can't just look it up.
            book_idxs = tuple(filter(lambda x: x != book_idx, book_idxs))
            yield {
                "len_book_idxs": len(book_idxs),
                "book_idxs": book_idxs,
                "book_idx": book_idx,
                "label": rating,
            }
            if rating == 1:
                # Generate a random negative book_id not in book_idxs.
                random_negative = np.random.randint(0, n_books)
                while random_negative in book_idxs:
                    random_negative = np.random.randint(0, n_books)
                yield {
                    "len_book_idxs": len(book_idxs),
                    "book_idxs": book_idxs,
                    "book_idx": random_negative,
                    "label": 0,
                }

    return generator


def get_data_tensors(df):
    # Use generator to get examples each epoch, along with shuffling and batching.
    padded_shapes = {
        "len_book_idxs": [],
        "book_idxs": [None],
        "book_idx": [],
        "label": [],
    }
    dataset = (
        tf.data.Dataset.from_generator(
            get_examples_generator(df), {k: tf.int64 for k in padded_shapes}
        )
        .shuffle(123)
        .repeat(None)
        .padded_batch(batch_size=256, padded_shapes=padded_shapes)
    )
    tensor_dict = tf.compat.v1.data.make_one_shot_iterator(dataset).get_next()
    return (
        tensor_dict["len_book_idxs"],
        tensor_dict["book_idxs"],
        tensor_dict["book_idx"],
        tensor_dict["label"],
    )

We now train the model on our combined training data (data labeled by LFs plus dev data).


In [13]:
model = get_model()

train_data_tensors = get_data_tensors(combined_df_train)
val_data_tensors = get_data_tensors(df_val)
model.fit(
    train_data_tensors[:-1],
    train_data_tensors[-1],
    steps_per_epoch=300,
    validation_data=(val_data_tensors[:-1], val_data_tensors[-1]),
    validation_steps=40,
    epochs=30,
    verbose=1,
)

W0811 03:42:43.977466 140444578916160 deprecation.py:506] From /home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


W0811 03:42:44.057152 140444578916160 deprecation.py:506] From /home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


W0811 03:42:44.172415 140444578916160 deprecation.py:323] From /home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


W0811 03:42:44.194866 140444578916160 deprecation.py:323] From /home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:494: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


W0811 03:42:44.384439 140444578916160 deprecation.py:506] From /home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/keras/optimizer_v2/adagrad.py:105: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

293/300 [============================>.] - ETA: 0s - loss: 0.5910 - acc: 0.7120 - f1: 0.4614 - precision: 0.8576 - recall: 0.3208

295/300 [============================>.] - ETA: 0s - loss: 0.5911 - acc: 0.7116 - f1: 0.4608 - precision: 0.8568 - recall: 0.3203

297/300 [============================>.] - ETA: 0s - loss: 0.5914 - acc: 0.7116 - f1: 0.4608 - precision: 0.8563 - recall: 0.3203

299/300 [============================>.] - ETA: 0s - loss: 0.5916 - acc: 0.7113 - f1: 0.4604 - precision: 0.8552 - recall: 0.3200

300/300 [==============================] - 13s 42ms/step - loss: 0.5918 - acc: 0.7112 - f1: 0.4604 - precision: 0.8547 - recall: 0.3201 - val_loss: 0.6519 - val_acc: 0.6502 - val_f1: 0.3609 - val_precision: 0.6148 - val_recall: 0.2670


Finally, we evaluate the model's predicted ratings on our test data.


In [14]:
test_data_tensors = get_data_tensors(df_test)
model.evaluate(test_data_tensors[:-1], test_data_tensors[-1], steps=30)

 1/30 [>.............................] - ETA: 3s - loss: 0.6254 - acc: 0.7031 - f1: 0.5529 - precision: 0.6620 - recall: 0.4747

 3/30 [==>...........................] - ETA: 1s - loss: 0.6311 - acc: 0.6940 - f1: 0.5326 - precision: 0.7258 - recall: 0.4242

 5/30 [====>.........................] - ETA: 1s - loss: 0.6374 - acc: 0.6687 - f1: 0.4564 - precision: 0.6851 - recall: 0.3497

 7/30 [======>.......................] - ETA: 0s - loss: 0.6434 - acc: 0.6518 - f1: 0.4103 - precision: 0.6825 - recall: 0.3038

 9/30 [========>.....................] - ETA: 0s - loss: 0.6453 - acc: 0.6519 - f1: 0.4068 - precision: 0.6965 - recall: 0.2959

...

24/30 [=======================>......] - ETA: 0s - loss: 0.6446 - acc: 0.6559 - f1: 0.4074 - precision: 0.6953 - recall: 0.2969

26/30 [=========================>....] - ETA: 0s - loss: 0.6438 - acc: 0.6562 - f1: 0.3981 - precision: 0.6683 - recall: 0.2931

28/30 [===========================>..] - ETA: 0s - loss: 0.6428 - acc: 0.6575 - f1: 0.3997 - precision: 0.6675 - recall: 0.2949

30/30 [==============================] - 1s 31ms/step - loss: 0.6434 - acc: 0.6542 - f1: 0.3968 - precision: 0.6623 - recall: 0.2922


[0.6434326887130737, 0.65416664, 0.39681196, 0.6622598, 0.29224512]

## Summary

In this tutorial, we showed one way to use Snorkel for recommendations. We used books' metadata and review text to create `LF`s that estimate user ratings. We used a `LabelModel` to combine the outputs of those `LF`s. Finally, we trained a model to predict what books a user will read and like (and therefore what books should be recommended to the user) based only on what books the user has interacted with in the past.

Here we demonstrated one way to use Snorkel for training a recommender system. Note, however, that this approach could easily be adapted to take advantage of additional information as it is available (e.g., user profile data, denser user ratings, and so on.)